# IMPORTING THE LIBRARIES 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.pyplot as plt1 

import statistics
import os

###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.cluster import KMeans

###############################################
from bokeh.plotting import figure , show
from pybaselines import whittaker as pl



from bokeh.io import output_notebook
output_notebook()

# DATA ARRANGEMENT

In the main directory we can see that there are 8 subfolders. 

Each subfolder have almost 12 spectras per sample , the idea behind that would be , instead of having just one spectra per sample , and to just rely on one information , its always better have to multiple measurements per samples , and then this could be used for building the Calibration Model

Instead of having 12 different csv per samples , its always good to have a single dataframe -> This new dataframe will have 1st column as wavelength , and 2nd -13th column as Intensities 

In [ ]:
def load_data(folder_path):
    # List to store DataFrames for intensity columns
    intensity_dfs = []

    # List to store file names
    file_names = []

    # Get a list of .txt files in the folder
    txt_files = [file_name for file_name in os.listdir(folder_path) if file_name.endswith('.txt')]

    # Sort the .txt files based on their numerical order
    # txt_files.sort(key=lambda x: int(re.search(r'_(\d+)\.txt', x).group(1)))

    # Read the wavelength values from the first file
    first_file_path = os.path.join(folder_path, txt_files[0])
    wavelength_df = pd.read_csv(first_file_path, header=None, delimiter=';', usecols=[0], names=['wavelength'])

    # Loop through each file in ascending order
    for file_name in txt_files:
        file_path = os.path.join(folder_path, file_name)
        
        # Read intensity values from each file into a DataFrame
        intensity_df = pd.read_csv(file_path, header=None, delimiter=';', usecols=[1], names=['intensity'])
        
        # Store intensity DataFrame
        intensity_dfs.append(intensity_df)
        
        # Store file name
        file_names.append(os.path.splitext(file_name)[0])

    # Concatenate intensity DataFrames
    result_df = pd.concat(intensity_dfs, axis=1)

    # Add the wavelength column to the result DataFrame
    result_df = pd.concat([wavelength_df, result_df], axis=1)

    # Rename the columns with file names
    result_df.columns = ['wavelength'] + file_names

    return result_df


Loading the dataframe.

In [ ]:
SAMPLE1_Raw_df = load_data('Batch_2/sample_1')
###################################################
SAMPLE2_Raw_df = load_data('Batch_2/sample_2')
###################################################
SAMPLE3_Raw_df = load_data('Batch_2/sample_3')
###################################################
SAMPLE4_Raw_df = load_data('Batch_2/sample_4')
###################################################
SAMPLE5_Raw_df = load_data('Batch_2/sample_5')
###################################################
SAMPLE6_Raw_df = load_data('Batch_2/sample_6')
###################################################
SAMPLE7_Raw_df = load_data('Batch_2/sample_7')
####################################################
SAMPLE8_Raw_df = load_data('Batch_2/sample_8')
####################################################
SAMPLE9_Raw_df = load_data('Batch_3/sample_9')
####################################################
SAMPLE10_Raw_df = load_data('Batch_3/sample_10')
####################################################
SAMPLE11_Raw_df = load_data('Batch_3/sample_11')
####################################################
SAMPLE12_Raw_df = load_data('Batch_3/sample_12')
####################################################
SAMPLE13_Raw_df = load_data('Batch_3/sample_13')
####################################################
SAMPLE14_Raw_df = load_data('Batch_3/sample_14')
####################################################
SAMPLE15_Raw_df = load_data('Batch_3/sample_15')
####################################################
SAMPLE16_Raw_df = load_data('Batch_3/sample_16')
####################################################
SAMPLE17_Raw_df = load_data('Batch_3/sample_17')
####################################################

SAMPLE1_Raw_df

# Peak Selection and Data Trimming

The dataframe  which we have is very big ,it could be trimmed now according to the wavelength , by adjusting two parameters "Wavelength_Min" , "Wavelength_Max"

In [ ]:


#Getting the full wavelength
Wavelength_Min = 230 #200.041162 
Wavelength_Max = 540 #963.321041 


SAMPLE1_Select_df = SAMPLE1_Raw_df[(SAMPLE1_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE1_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE2_Select_df = SAMPLE2_Raw_df[(SAMPLE2_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE2_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE3_Select_df = SAMPLE3_Raw_df[(SAMPLE3_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE3_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE4_Select_df = SAMPLE4_Raw_df[(SAMPLE4_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE4_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE5_Select_df = SAMPLE5_Raw_df[(SAMPLE5_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE5_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE6_Select_df = SAMPLE6_Raw_df[(SAMPLE6_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE6_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE7_Select_df = SAMPLE7_Raw_df[(SAMPLE7_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE7_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE8_Select_df = SAMPLE8_Raw_df[(SAMPLE8_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE8_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE9_Select_df = SAMPLE9_Raw_df[(SAMPLE9_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE9_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE10_Select_df = SAMPLE10_Raw_df[(SAMPLE10_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE10_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE11_Select_df = SAMPLE11_Raw_df[(SAMPLE11_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE11_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE12_Select_df = SAMPLE12_Raw_df[(SAMPLE12_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE12_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE13_Select_df = SAMPLE13_Raw_df[(SAMPLE13_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE13_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE14_Select_df = SAMPLE14_Raw_df[(SAMPLE14_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE14_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE15_Select_df = SAMPLE15_Raw_df[(SAMPLE15_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE15_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE16_Select_df = SAMPLE16_Raw_df[(SAMPLE16_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE16_Raw_df['wavelength'] <= Wavelength_Max)]
SAMPLE17_Select_df = SAMPLE17_Raw_df[(SAMPLE17_Raw_df['wavelength'] >= Wavelength_Min) & (SAMPLE17_Raw_df['wavelength'] <= Wavelength_Max)]


SAMPLE1_Select_df


Lets plot a line plot ,to get a better picture 

In [ ]:
# Selected_df_Plot = figure(title = 'Selected Data Plot' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

# Selected_df_Plot.line(SAMPLE1_Select_df.wavelength,SAMPLE1_Select_df.sample1_6 , line_width = 2, color ="blue" )
# #####################################################################################################################
# Selected_df_Plot.line(SAMPLE2_Select_df.wavelength,SAMPLE2_Select_df.sample2_6 , line_width = 2, color ="orange" )
# ###############################################################################################################
# Selected_df_Plot.line(SAMPLE3_Select_df.wavelength,SAMPLE3_Select_df.sample3_6 , line_width = 2, color ="green")
# ######################################################################################################################
# Selected_df_Plot.line(SAMPLE4_Select_df.wavelength,SAMPLE4_Select_df.sample4_6 , line_width = 2, color ="red")
# ##############################################################################################################
# Selected_df_Plot.line(SAMPLE5_Select_df.wavelength,SAMPLE5_Select_df.sample5_6 , line_width = 2, color ="purple")
# ###############################################################################################################
# Selected_df_Plot.line(SAMPLE6_Select_df.wavelength,SAMPLE6_Select_df.sample6_6 , line_width = 2, color ="brown")
# ####################################################################################################################
# Selected_df_Plot.line(SAMPLE7_Select_df.wavelength,SAMPLE7_Select_df.sample7_6 , line_width = 2, color ="pink")
# ##################################################################################################################
# Selected_df_Plot.line(SAMPLE8_Select_df.wavelength,SAMPLE8_Select_df.sample8_6 , line_width = 2, color ="gray")
# ###################################################################################################################
# Selected_df_Plot.line(SAMPLE9_Select_df.wavelength,SAMPLE9_Select_df.sample9_6 , line_width = 2, color ="olive")
# ###################################################################################################################
# Selected_df_Plot.line(SAMPLE10_Select_df.wavelength,SAMPLE10_Select_df.sample10_6 , line_width = 2, color ="Cyan")
# ###################################################################################################################
# Selected_df_Plot.line(SAMPLE11_Select_df.wavelength,SAMPLE11_Select_df.sample11_6 , line_width = 2, color ="black")
# ###################################################################################################################
# Selected_df_Plot.line(SAMPLE12_Select_df.wavelength,SAMPLE12_Select_df.sample12_6 , line_width = 2, color ="tomato")
# ###################################################################################################################
# Selected_df_Plot.line(SAMPLE13_Select_df.wavelength,SAMPLE13_Select_df.sample13_6 , line_width = 2, color ="steelblue")
# ###################################################################################################################
# Selected_df_Plot.line(SAMPLE14_Select_df.wavelength,SAMPLE14_Select_df.sample14_6 , line_width = 2, color ="limegreen")
# ###################################################################################################################
# Selected_df_Plot.line(SAMPLE15_Select_df.wavelength,SAMPLE15_Select_df.sample15_6 , line_width = 2, color ="deeppink")
# ###################################################################################################################
# Selected_df_Plot.line(SAMPLE16_Select_df.wavelength,SAMPLE16_Select_df.sample16_6 , line_width = 2, color ="gold")
# ###################################################################################################################
# Selected_df_Plot.line(SAMPLE17_Select_df.wavelength,SAMPLE17_Select_df.sample17_6 , line_width = 2, color ="indigo")
# ###################################################################################################################

# Selected_df_Plot.width = 900
# Selected_df_Plot.height = 500
# show(Selected_df_Plot)

# Data Preprocessing of the Spectra

In [ ]:
def baseline_correction(df):
    """
    Perform baseline correction on the intensity columns of the input DataFrame and create a new DataFrame with corrected values.
    
    Parameters:
        df (DataFrame): Input DataFrame containing the wavelength and intensity columns.
        
    Returns:
        DataFrame: New DataFrame with baseline-corrected intensity columns and the same wavelength column as the input DataFrame.
    """
    # Copy the 'wavelength' column from the input DataFrame
    new_df = pd.DataFrame({'wavelength': df['wavelength']})
    
    # Perform baseline correction for each intensity column and add them to the new DataFrame
    for col in df.columns[1:]:  # Exclude the 'wavelength' column
        baseline, _ = pl.asls(df[col])
        corrected_values = df[col] - baseline
        new_df[col] = corrected_values
    
    return new_df

The above plot eventhough a spectra , is still a Raw Spectra , which still has lot of Artifects , before proceeding for the Univariate Calibration , its important to Pre Process the Raw Spectra accordingly. Various Pre Processing Techniques could be used here :- 

1) Baseline Correction - Very Very little  background radiation is still present in the spectra, which corresponds to the spectral baseline and imposes difficulties for quantitative elemental analysis.

In [ ]:
SAMPLE1_BaselineCorrected_df = baseline_correction(SAMPLE1_Select_df)
SAMPLE2_BaselineCorrected_df = baseline_correction(SAMPLE2_Select_df)
SAMPLE3_BaselineCorrected_df = baseline_correction(SAMPLE3_Select_df)
SAMPLE4_BaselineCorrected_df = baseline_correction(SAMPLE4_Select_df)
SAMPLE5_BaselineCorrected_df = baseline_correction(SAMPLE5_Select_df)
SAMPLE6_BaselineCorrected_df = baseline_correction(SAMPLE6_Select_df)
SAMPLE7_BaselineCorrected_df = baseline_correction(SAMPLE7_Select_df)
SAMPLE8_BaselineCorrected_df = baseline_correction(SAMPLE8_Select_df)
SAMPLE9_BaselineCorrected_df = baseline_correction(SAMPLE9_Select_df)
SAMPLE10_BaselineCorrected_df = baseline_correction(SAMPLE10_Select_df)
SAMPLE11_BaselineCorrected_df = baseline_correction(SAMPLE11_Select_df)
SAMPLE12_BaselineCorrected_df = baseline_correction(SAMPLE12_Select_df)
SAMPLE13_BaselineCorrected_df = baseline_correction(SAMPLE13_Select_df)
SAMPLE14_BaselineCorrected_df = baseline_correction(SAMPLE14_Select_df)
SAMPLE15_BaselineCorrected_df = baseline_correction(SAMPLE15_Select_df)
SAMPLE16_BaselineCorrected_df = baseline_correction(SAMPLE16_Select_df)
SAMPLE17_BaselineCorrected_df = baseline_correction(SAMPLE17_Select_df)

print(SAMPLE1_BaselineCorrected_df)

mean_intensity_1 = SAMPLE1_BaselineCorrected_df['sample1_1'].mean()
std_intensity_1= SAMPLE1_BaselineCorrected_df['sample1_1'].std()

print(mean_intensity_1)
print(std_intensity_1)

In [ ]:
# Baseline_Correction_Plot = figure(title = 'Baseline Correction' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

# Baseline_Correction_Plot.line(SAMPLE4_Select_df.wavelength,SAMPLE4_Select_df.sample4_1 , line_width = 2, color ="red" )
# Baseline_Correction_Plot.line(SAMPLE4_BaselineCorrected_df.wavelength,SAMPLE4_BaselineCorrected_df.sample4_1 , line_width = 2, color ="green" )


# Baseline_Correction_Plot.width = 900
# Baseline_Correction_Plot.height = 500
# show(Baseline_Correction_Plot)

2) Normalization - Its usually noticed that , for a measurement of a similar sample ,there is a lot of scattering in the intensities , this typical artifect is called scattering.

For Instance , you could see the plot above , Though this are the plots from the same sample SAMPLE1 , measured on 12 different Areas , its quite visible , for some  spectras , the peak heights or Intensities are not same.


In [ ]:

# def standard_normal_variate_normalization(df):
#     # Selecting only the intensity columns (assuming the first column is wavelength)
#     intensities = df.iloc[:, 1:]

#     # Calculating mean and standard deviation for each column
#     means = intensities.mean(axis=0)
#     stds = intensities.std(axis=0)

#     # Applying standard normal variate normalization column-wise
#     normalized_intensities = (intensities - means) / stds

#     # Combining the wavelength column with normalized intensities
#     normalized_df = pd.concat([df.iloc[:, 0], normalized_intensities], axis=1)

#     # Calculating mean and standard deviation for each normalized column
#     normalized_means = normalized_intensities.mean(axis=0)
#     normalized_stds = normalized_intensities.std(axis=0)

#     return normalized_df


In [ ]:
def standard_normal_variate_normalization(df):
    # Select columns containing intensities
    intensity_cols = df.columns[1:]  #  columns 2 to 13 are intensities and are stored in intensity_cols

    # SVN normalization
    for col in intensity_cols:
        mean_intensity = df[col].mean()
        std_intensity = df[col].std()
        df[col] = (df[col] - mean_intensity) / std_intensity

    return df

In [ ]:

def total_intensity_normalization(df):
    # Selecting only the intensity columns (assuming the first column is wavelength)
    intensities = df.iloc[:, 1:]  

    # Initialize an empty DataFrame to store normalized intensities
    normalized_intensities = pd.DataFrame()

    # Iterate over each column (each spectrum)
    for column in intensities.columns:
        spectrum = intensities[column]
        total_intensity = spectrum.sum()
        
        # Normalize the spectrum by dividing each intensity value by the total intensity
        normalized_spectrum = spectrum / total_intensity
        
        # Append the normalized spectrum to the DataFrame of normalized intensities
        normalized_intensities[column] = normalized_spectrum

    # Combine the wavelength column with normalized intensities
    normalized_df = pd.concat([df.iloc[:, 0], normalized_intensities], axis=1)

    return normalized_df

In [ ]:

def unit_norm_normalization(df):
    # Selecting only the intensity columns (assuming the first column is wavelength)
    intensities_1 = df.iloc[:, 1:]  
    
    # Initialize an empty DataFrame to store normalized intensities
    normalized_intensities_1 = pd.DataFrame()

    # Iterate over each column (each spectrum)
    for column_1 in intensities_1.columns:
        spectrum_1 = intensities_1[column_1]
        
        # Calculate the norm of the spectrum using a simple equation
        spectrum_norm = np.sqrt(np.sum(spectrum_1 ** 2))
        
        # Normalize the spectrum by dividing each intensity value by its norm
        normalized_spectrum_1 = spectrum_1 / spectrum_norm
        
        # Assign the normalized spectrum to the DataFrame of normalized intensities
        normalized_intensities_1[column_1] = normalized_spectrum_1

    # Combine the wavelength column with normalized intensities
    normalized_df_1 = pd.concat([df.iloc[:, 0], normalized_intensities_1], axis=1)
    
    return normalized_df_1


In [ ]:

def max_intensity_normalization(df):
    # Selecting only the intensity columns (assuming the first column is wavelength)
    intensities_2 = df.iloc[:, 1:]  

    # Initialize an empty DataFrame to store normalized intensities
    normalized_intensities_2 = pd.DataFrame()

    # Iterate over each column (each spectrum)
    for column_2 in intensities_2.columns:
        spectrum_2 = intensities_2[column_2]
        
        # Find the maximum intensity value in the spectrum
        max_intensity = spectrum_2.max()
        
        # Normalize the spectrum by dividing each intensity value by the maximum intensity
        normalized_spectrum_2 = spectrum_2 / max_intensity
        
        # Assign the normalized spectrum to the DataFrame of normalized intensities
        normalized_intensities_2[column_2] = normalized_spectrum_2

    # Combine the wavelength column with normalized intensities
    normalized_df_2 = pd.concat([df.iloc[:, 0], normalized_intensities_2], axis=1)

    return normalized_df_2


In [ ]:
SAMPLE1_Normalized_df = standard_normal_variate_normalization(SAMPLE1_BaselineCorrected_df)
SAMPLE2_Normalized_df = standard_normal_variate_normalization(SAMPLE2_BaselineCorrected_df)
SAMPLE3_Normalized_df = standard_normal_variate_normalization(SAMPLE3_BaselineCorrected_df)
SAMPLE4_Normalized_df = standard_normal_variate_normalization(SAMPLE4_BaselineCorrected_df)
SAMPLE5_Normalized_df = standard_normal_variate_normalization(SAMPLE5_BaselineCorrected_df)
SAMPLE6_Normalized_df = standard_normal_variate_normalization(SAMPLE6_BaselineCorrected_df)
SAMPLE7_Normalized_df = standard_normal_variate_normalization(SAMPLE7_BaselineCorrected_df)
SAMPLE8_Normalized_df = standard_normal_variate_normalization(SAMPLE8_BaselineCorrected_df)
SAMPLE9_Normalized_df = standard_normal_variate_normalization(SAMPLE9_BaselineCorrected_df)
SAMPLE10_Normalized_df = standard_normal_variate_normalization(SAMPLE10_BaselineCorrected_df)
SAMPLE11_Normalized_df = standard_normal_variate_normalization(SAMPLE11_BaselineCorrected_df)
SAMPLE12_Normalized_df = standard_normal_variate_normalization(SAMPLE12_BaselineCorrected_df)
SAMPLE13_Normalized_df = standard_normal_variate_normalization(SAMPLE13_BaselineCorrected_df)
SAMPLE14_Normalized_df = standard_normal_variate_normalization(SAMPLE14_BaselineCorrected_df)
SAMPLE15_Normalized_df = standard_normal_variate_normalization(SAMPLE15_BaselineCorrected_df)
SAMPLE16_Normalized_df = standard_normal_variate_normalization(SAMPLE16_BaselineCorrected_df)
SAMPLE17_Normalized_df = standard_normal_variate_normalization(SAMPLE17_BaselineCorrected_df)

mean_intensity = SAMPLE1_Normalized_df['sample1_1'].mean()
std_intensity = SAMPLE1_Normalized_df['sample1_1'].std()

# print(mean_intensity)
# print(std_intensity)

SAMPLE17_Normalized_df

In [ ]:
import pandas as pd

# Assuming you have 17 DataFrames stored in a list called `dfs`
# Replace this with your actual list of DataFrames
all_normalized_df = [SAMPLE1_Normalized_df, SAMPLE2_Normalized_df, SAMPLE3_Normalized_df, SAMPLE4_Normalized_df, SAMPLE5_Normalized_df, SAMPLE6_Normalized_df, SAMPLE7_Normalized_df, SAMPLE8_Normalized_df, SAMPLE9_Normalized_df, SAMPLE10_Normalized_df, SAMPLE11_Normalized_df, SAMPLE12_Normalized_df, SAMPLE13_Normalized_df, SAMPLE14_Normalized_df, SAMPLE15_Normalized_df, SAMPLE16_Normalized_df, SAMPLE17_Normalized_df]

# Extract the wavelength column from the first DataFrame
wavelength_df = all_normalized_df[0][['wavelength']]
# print(wavelength_df)

# Concatenate all DataFrames (excluding the 'wavelength' column)
combined_df = pd.concat([df.drop(columns='wavelength') for df in all_normalized_df], axis=1)


# Add the wavelength column back
combined_df = pd.concat([wavelength_df, combined_df], axis=1)

combined_df = combined_df.dropna()

preprocessed_df = combined_df


# Display the combined DataFrame
print(combined_df)

# Optionally, save the combined DataFrame to a CSV file
# combined_df.to_csv('combined_samples.csv', index=False)


In [ ]:
#Average the Resultant Spectra

In [ ]:
# def average_intensity(df, intensity_name):
#     # Select intensity columns (from column 2 to column 13)
#     intensity_columns = df.columns[1:]

#     # Calculate the mean of intensity columns
#     averaged_intensity = df[intensity_columns].mean(axis=1)

#     # Create a new DataFrame with wavelength and averaged intensity
#     averaged_df = pd.DataFrame({'wavelength': df['wavelength'], intensity_name: averaged_intensity})

#     return averaged_df

In [ ]:
# SAMPLE1_Averaged_df = average_intensity(df = SAMPLE1_Normalized_df , intensity_name= 'sample1')
# SAMPLE2_Averaged_df = average_intensity(df = SAMPLE2_Normalized_df , intensity_name= 'sample2')
# SAMPLE3_Averaged_df = average_intensity(df = SAMPLE3_Normalized_df , intensity_name= 'sample3')
# SAMPLE4_Averaged_df = average_intensity(df = SAMPLE4_Normalized_df , intensity_name= 'sample4')
# SAMPLE5_Averaged_df = average_intensity(df = SAMPLE5_Normalized_df , intensity_name= 'sample5')
# SAMPLE6_Averaged_df = average_intensity(df = SAMPLE6_Normalized_df , intensity_name= 'sample6')
# SAMPLE7_Averaged_df = average_intensity(df = SAMPLE7_Normalized_df , intensity_name= 'sample7')
# SAMPLE8_Averaged_df = average_intensity(df = SAMPLE8_Normalized_df , intensity_name= 'sample8')
# SAMPLE9_Averaged_df = average_intensity(df = SAMPLE9_Normalized_df , intensity_name= 'sample9')
# SAMPLE10_Averaged_df = average_intensity(df = SAMPLE10_Normalized_df , intensity_name= 'sample10')
# SAMPLE11_Averaged_df = average_intensity(df = SAMPLE11_Normalized_df , intensity_name= 'sample11')
# SAMPLE12_Averaged_df = average_intensity(df = SAMPLE12_Normalized_df , intensity_name= 'sample12')
# SAMPLE13_Averaged_df = average_intensity(df = SAMPLE13_Normalized_df , intensity_name= 'sample13')
# SAMPLE14_Averaged_df = average_intensity(df = SAMPLE14_Normalized_df , intensity_name= 'sample14')
# SAMPLE15_Averaged_df = average_intensity(df = SAMPLE15_Normalized_df , intensity_name= 'sample15')
# SAMPLE16_Averaged_df = average_intensity(df = SAMPLE16_Normalized_df , intensity_name= 'sample16')
# SAMPLE17_Averaged_df = average_intensity(df = SAMPLE17_Normalized_df , intensity_name= 'sample17')

# SAMPLE1_Averaged_df

In [ ]:
# # List of DataFrame names
# data_frames = [SAMPLE1_Averaged_df, SAMPLE2_Averaged_df, SAMPLE3_Averaged_df, 
#                SAMPLE4_Averaged_df, SAMPLE5_Averaged_df, SAMPLE6_Averaged_df, 
#                SAMPLE7_Averaged_df, SAMPLE8_Averaged_df ,SAMPLE9_Averaged_df,
#                SAMPLE10_Averaged_df,SAMPLE11_Averaged_df,SAMPLE12_Averaged_df,
#                SAMPLE13_Averaged_df,SAMPLE14_Averaged_df,SAMPLE15_Averaged_df,
#                SAMPLE16_Averaged_df,SAMPLE17_Averaged_df]

# # Initialize the new DataFrame with the 'wavelength' column from the first DataFrame
# preprocessed_df = data_frames[0][['wavelength']].copy()

# # Add 'sample' columns from each DataFrame
# for i, df in enumerate(data_frames):
#     sample_column = f'sample{i+1}'
#     preprocessed_df[sample_column] = df[sample_column]

# # Display the new DataFrame
# preprocessed_df

In [ ]:
# Preprocessed_df_Plot = figure(title = 'Preprocessed Data Plot' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample1 , line_width = 2, color ="blue" )
# #####################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample2 , line_width = 2, color ="orange" )
# ###############################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample3 , line_width = 2, color ="green")
# ######################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample4 , line_width = 2, color ="red")
# ##############################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample5 , line_width = 2, color ="purple")
# ###############################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample6 , line_width = 2, color ="brown")
# ####################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample7 , line_width = 2, color ="pink")
# ##################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample8 , line_width = 2, color ="gray")
# ###################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample9 , line_width = 2, color ="olive")
# ###################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample10 , line_width = 2, color ="Cyan")
# ###################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample11 , line_width = 2, color ="black")
# ###################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample12 , line_width = 2, color ="tomato")
# ###################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample13 , line_width = 2, color ="steelblue")
# ###################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample14 , line_width = 2, color ="limegreen")
# ###################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample15 , line_width = 2, color ="deeppink")
# ###################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample16 , line_width = 2, color ="gold")
# ###################################################################################################################
# Preprocessed_df_Plot.line(preprocessed_df.wavelength,preprocessed_df.sample17 , line_width = 2, color ="indigo")
# ###################################################################################################################

# Preprocessed_df_Plot.width = 900
# Preprocessed_df_Plot.height = 500
# show(Preprocessed_df_Plot)


In [ ]:

preprocessed_df_trasnposed = preprocessed_df.T
print(preprocessed_df_trasnposed)

########################################

labels = preprocessed_df_trasnposed[1:].index.tolist()  #collecting the labels from the samples
print(labels)

############################################

selected_samples = preprocessed_df_trasnposed.loc['sample9_9':'sample17_10']
X = selected_samples.values


selected_samples


In [ ]:
#Determining the Number of Principle Component Required

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt1



pca = PCA(n_components=17)

# Perform PCA
X_pca = pca.fit_transform(X)

# Explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_  # Access from the PCA object

percentage_list = []
for ite in explained_variance_ratio:
    percentage = ite * 100
    percentage_list.append(percentage)

print(percentage_list)


# PCA components
pca_components = pca.components_

# Plot the explained variance ratio
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, 'bo-', linewidth=2)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance Ratio by Principal Component')
plt.savefig('Explained_Variance_batch3.png', bbox_inches='tight', dpi=300)
plt.show()


# Create a bar plot
plt1.figure(figsize=(10, 6))
plt1.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, color='skyblue')
plt1.xlabel('Principal Component')
plt1.ylabel('Explained Variance Ratio')
plt1.title('Explained Variance Ratio by Principal Component')
plt1.xticks(range(1, len(explained_variance_ratio) + 1))  # Ensure x-ticks are labeled correctly
plt.savefig('Explained_Variance_bargraph_batch3.png', bbox_inches='tight', dpi=300)
plt1.show()



In [ ]:
# import pandas as pd
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt2

# # Assuming preprocessed_df_trasnposed and X are already defined as per your provided code

# # Initialize PCA with 2 components for 2D plot
# pca_2d = PCA(n_components=4)

# # Perform PCA to reduce to 2 dimensions
# X_pca_2d = pca_2d.fit_transform(X)

# # Create a DataFrame for easy plotting
# df_pca_2d = pd.DataFrame(X_pca_2d, columns=['PC1', 'PC2' , 'PC3' , 'PC4'], index=selected_samples.index)

# # Plot the 2D PCA results
# plt.figure(figsize=(10, 8))

# # Plot each sample with its respective label
# plt.scatter(df_pca_2d['PC1'], df_pca_2d['PC3'], c='blue', marker='x')

# # Adding labels to the plot
# for i, label in enumerate(df_pca_2d.index):
#     plt.text(df_pca_2d['PC1'][i], df_pca_2d['PC3'][i], label, fontsize=5)

# plt.xlabel('Principal Component 1')
# plt.ylabel('Principal Component 2')
# plt.title('2D PCA of the Data')
# plt.grid(True)
# plt.show()


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Initialize PCA with 2 components
pca_2d = PCA(n_components=3)

# Perform PCA to reduce to 2 dimensions
X_pca_2d = pca_2d.fit_transform(X)

# Create a DataFrame for easy plotting
df_pca_2d = pd.DataFrame(X_pca_2d, columns=['PC1', 'PC2' , 'PC3'], index=selected_samples.index)

# Define markers for different sample groups
markers = {
    'sample1': 'o',
    'sample2': '^',
    'sample3': 's',
    'sample4': 'p',
    'sample5': '*',
    'sample6': 'D',
    'sample7': 'H',
    'sample8': '+',
    'sample9': 'x',
    'sample10': '|',
    'sample11': '_',
    'sample12': '1',
    'sample13': '2',
    'sample14': '3',
    'sample15': '4',
    'sample16': 'v',
    'sample17': '<'
}

# Extract the base name of each sample to determine its group
sample_groups = {}
for sample in df_pca_2d.index:
    base_name = sample.split('_')[0]  # Get the base name like 'sample1', 'sample2', etc.
    if base_name in markers:
        if base_name not in sample_groups:
            sample_groups[base_name] = []
        sample_groups[base_name].append(sample)

# Plot the 2D PCA results
fig, ax = plt.subplots(figsize=(10, 8))

# Plot each group with a different marker
for group, samples in sample_groups.items():
    marker = markers[group]  # Use the assigned marker for this group
    group_df = df_pca_2d.loc[samples]
    ax.scatter(group_df['PC2'], group_df['PC3'], 
               c=np.random.rand(3,),  # Random color for each group
               marker=marker, s=50, edgecolor='k', label=group)

# Add a legend to show which marker corresponds to which sample group
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc='upper left')


# Set labels and title with better font and style
ax.set_xlabel('Principal Component 2', fontsize=14, fontweight='bold')
ax.set_ylabel('Principal Component 3', fontsize=14, fontweight='bold')
ax.set_title('2D PCA of the Data', fontsize=16, fontweight='bold')


# Customize the grid and background
ax.grid(True, linestyle='--', alpha=0.6)

plt.savefig('pca_batch3_pc2_pc3.png', bbox_inches='tight', dpi=300)

plt.show()


In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Initialize PCA with 3 components
pca_3d = PCA(n_components=3)

# Perform PCA to reduce to 3 dimensions
X_pca_3d = pca_3d.fit_transform(X)

# Create a DataFrame for easy plotting
df_pca_3d = pd.DataFrame(X_pca_3d, columns=['PC1', 'PC2', 'PC3'], index=selected_samples.index)

# Define markers for different sample groups
# Here we're using sample categories like 'sample1', 'sample2', etc.
# You might need to adjust this according to your actual sample groupings.
markers = {
    'sample1': 'o',
    'sample2': '^',
    'sample3': 's',
    'sample4': 'p',
    'sample5': '*',
    'sample6': 'D',
    'sample7': 'H',
    'sample8': '+',
    'sample9': 'x',
    'sample10': '|',
    'sample11': '_',
    'sample12': '1',
    'sample13': '2',
    'sample14': '3',
    'sample15': '4',
    'sample16': 'v',
    'sample17': '<'
}



# Plot the 3D PCA results
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Extract the base name of each sample to determine its group
sample_groups = {}
for sample in df_pca_3d.index:
    base_name = sample.split('_')[0]  # Get the base name like 'sample1', 'sample2', etc.
    if base_name in markers:
        if base_name not in sample_groups:
            sample_groups[base_name] = []
        sample_groups[base_name].append(sample)

print(sample_groups)

# Plot each group with a different marker
for group, samples in sample_groups.items():
    marker = markers[group]  # Use the assigned marker for this group
    group_df = df_pca_3d.loc[samples]
    ax.scatter(group_df['PC1'], group_df['PC2'], group_df['PC3'], 
               c=np.random.rand(3,),  # Random color for each group
               marker=marker, s=50, edgecolor='k', label=group)

# Add a legend to show which marker corresponds to which sample group
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc='upper left')

# Set labels and title with better font and style
ax.set_xlabel('Principal Component 1', fontsize=14, fontweight='bold')
ax.set_ylabel('Principal Component 2', fontsize=14, fontweight='bold')
ax.set_zlabel('Principal Component 3', fontsize=14, fontweight='bold')
ax.set_title('3D PCA of the Data', fontsize=16, fontweight='bold')

# Customize the grid and background
ax.grid(True, linestyle='--', alpha=0.6)

plt.savefig('pca_batch3_pc1_pc2_pc3.png', bbox_inches='tight', dpi=300)
plt.show()
